In [8]:
!pip install langchain_groq
!pip install pypdf
!pip install chromadb
!pip install sentence_transformers
!pip install -U langchain-community
!pip install --upgrade gradio
!pip install --upgrade transformers sentence_transformers
!pip install gTTS
!pip install transformers
!pip install --upgrade tensorflow

In [ ]:
from gtts import gTTS
from transformers import pipeline
from langchain_groq import ChatGroq
from IPython import get_ipython
from IPython.display import display
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import random
import chromadb
import gradio as gr
import tempfile

stt_pipe = pipeline("automatic-speech-recognition", model="openai/whisper-small")

def initialize_llm():
    llm = ChatGroq(
        temperature=0.7,
        groq_api_key="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",
        model_name="llama-3.3-70b-versatile"
    )
    return llm

def create_vector_db():
    loader = DirectoryLoader(r"/content/data", glob='*.pdf', loader_cls=PyPDFLoader)
    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)

    embeddings=HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db=Chroma.from_documents(texts, embeddings, persist_directory='./chroma_db')
    vector_db.persist()
    return vector_db

def audio_to_text(audio_path):
    if audio_path is None:
        return None
    text = stt_pipe(audio_path)["text"]
    return text

def text_to_audio(text):
    if not text:
        return None
    tts = gTTS(text=text, lang='en')
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
    tts.save(temp_file.name)
    return temp_file.name

def process_user_input(user_input, chat_history):
    if not user_input.strip():
        return chat_history, None

    response = qa_chain({"query": user_input})
    answer = response["result"]
    response_variations = [
        f"✅ Great question! Here's what I think: {answer} 💡",
        f"🤔 Interesting! Here’s my take: {answer} What are your thoughts? 😊",
        f"📌 That’s a deep one! {answer} Have you experienced something like this before?",
        f"🎯 Boom! Here’s a quick insight: {answer} Need more details? Just ask! 😉",
    ]

    final_answer = random.choice(response_variations)

    crisis_keywords=['suicide', 'self-harm', 'kill myself', 'cutting', 'hurt myself', 'end my life',
    'no reason to live', 'want to die', 'jump off', 'overdose', 'slit my wrists',
    'hang myself', 'drown myself', 'hopeless', 'worthless', 'useless', 'no one cares',
    'empty', 'tired of life', 'can’t go on', 'nothing matters', 'lost all hope',
    'abused', 'molested', 'assaulted', 'raped', 'harassed', 'beaten', 'domestic violence',
    'forced', 'threatened', 'bullied', 'panic attack', 'can’t breathe', 'chest pain',
    'heart racing', 'shaking', 'dizzy', 'faint', 'feel like dying']
    if any(keyword in user_input.lower() for keyword in crisis_keywords):
        answer += """\n\nI'm really sorry you're feeling this way. You're not alone, and there are people who care about you. "
    "Please reach out to a trusted friend, family member, or a professional who can help. "
    "\n\nIf you're in crisis, please contact: "
    "\n- 🇮🇳 India: Vandrevala Foundation Helpline - 1860 266 2345 / Snehi - 91-9582208181 "
    "\n- 🇺🇸 USA: National Suicide Prevention Lifeline - 988 / Crisis Text Line - TEXT HOME to 741741 "
    "\n- 🇬🇧 UK: Samaritans - 116 123 "
    "\n- 🌍 International: Find a helpline near you at https://findahelpline.com/ "
    "\n\nIf you are in immediate danger, please seek emergency help or call 911/112."
\n\nI'm really sorry you're feeling this way. You're not alone, and there are people who care about you.
Please reach out to a trusted friend, family member, or a professional who can help.
\n\nIf you're in crisis, please contact:
\n- 🇮🇳 India: Vandrevala Foundation Helpline - 1860 266 2345 / Snehi - 91-9582208181
\n- 🇺🇺 USA: National Suicide Prevention Lifeline - 988 / Crisis Text Line - TEXT HOME to 741741
\n- 🇬🇧 UK: Samaritans - 116 123
\n- 🌍 International: Find a helpline near you at https://findahelpline.com/
\n\nIf you are in immediate danger, please seek emergency help or call 911/112.
💙 *You are valued and important.* Sometimes life feels overwhelming, but please remember that this moment will pass, and you will get through it. 💙
\n\n🌟 *It's okay to ask for help.* Everyone needs support sometimes, and reaching out is a brave step. You're doing the right thing by talking about it.
\n\n🌈 *You deserve to be heard.* If you’re feeling lost, don’t be afraid to let someone know what you’re going through. You are never a burden.
\n\n🤝 *You’re stronger than you realize.* It may feel impossible right now, but you have the strength to get through this. Take things one step at a time.
\n\n❤️ *There are people who care about you.* Sometimes, when it feels like nobody does, just know that someone out there is thinking of you right now.
\n\n🌟 *Recovery is a journey, and you don't have to walk it alone.* Support is just a call or message away.
\n\n🦋 *Even in the darkest moments, there’s hope.* Things may seem hard, but with time, things can get better. You have a future to look forward to.
\n\n📞 *Sometimes, reaching out can make a huge difference.* If you can, talk to someone you trust, whether that’s a friend, family member, or a professional.
\n\n⚡ *You are never a burden.* It’s okay to lean on others when you need to. Asking for help takes strength.
\n\n💜 *Take a deep breath.* Sometimes, just pausing and breathing can help clear your mind a little. I’m here with you.
\n\n🌻 *You are worthy of love, help, and healing.* There’s no shame in asking for help; it’s a courageous step toward better days.
\n\n💖 *Your life is precious.* Even when things seem impossible, there’s always a path forward. Don’t give up. People care about you more than you know.
\n\n💡 *When everything feels too much, it’s okay to take a break.* Try to step back, breathe, and reach out for support if you can.
\n\n🌈 *You’re not broken, you're going through something tough.* Tough times don’t define you, and they don’t last forever.
\n\n💫 *You matter.* You have a place in this world, and there’s always hope, even when it feels out of reach.
\n\n🌼 *It’s okay to be scared or unsure.* Feeling vulnerable doesn’t make you weak. You're facing challenges, but you're also capable of overcoming them.
\n\n🧡 *It’s okay to cry, to feel sad, and to express yourself.* Your emotions are valid, and they don’t make you any less strong.
\n\n💪 *You have the courage to reach out.* It’s not easy, but it’s the first step to healing. I’m proud of you for taking it.
\n\n❤️ *I’m here to listen.* You’re not bothering me, and you’re not alone. Your feelings matter, and they deserve to be heard.
\n\n🌸 *You don’t have to figure everything out right now.* Life can be overwhelming, but small steps toward healing are powerful too.
\n\n🌟 *There’s always support waiting for you.* It can feel hard to ask for help, but there are people who want to help you feel better.
\n\n💙 *You deserve peace and healing.* Take it one moment at a time, and remember that asking for help is a brave and important step.
\n\n💖 *If you need someone to talk to, I'm here.* You don’t have to carry everything alone—together, we can take steps to feel better.
\n\n🌺 *The first step is always the hardest.* Just talking about how you feel is an incredible strength. Keep going; you’re doing great.
\n\n💪 *Your resilience is amazing.* Even when it doesn’t feel like it, you’re making progress. Healing is a journey, and you’re on the right path.
\n\n✨ *It’s okay to take a break and rest.* Restoring yourself is part of the healing process.
\n\n💖 *Even on the tough days, you are doing your best.* There’s no rush, no expectations. Just take things one step at a time.
\n\n🌈 *I’m rooting for you.* You're doing the best you can, and that's enough. Healing takes time, but you’re not alone on this journey.
\n\n🦋 *Remember, your feelings are valid.* Whatever you're going through right now, it’s okay. Don’t hesitate to reach out for support.
\n\n💖 *Please know that this moment doesn’t define you.* You are valuable, and you have so much worth.
\n\n💜 *You’re not alone in this fight.* Reach out to someone who can offer the help and care you deserve. You don’t have to face it alone.
\n\n🌟 *I believe in you.* There is always a way forward, and you have what it takes to get there."""


    audio_path = text_to_audio(answer)
    chat_history.append((user_input, answer))
    return chat_history, audio_path

def gradio_chat(user_input, chat_history):
    chat_history, audio_path = process_user_input(user_input, chat_history)
    return "", chat_history, audio_path

def voice_chat(audio_path, chat_history):
    if audio_path is None:
        return chat_history, None

    user_input = audio_to_text(audio_path)
    if not user_input.strip():
        return chat_history + [("", "Could not understand the audio. Please try again.")], None

    chat_history, audio_path = process_user_input(user_input, chat_history)
    return chat_history, audio_path

llm = initialize_llm()

if not os.path.exists("./chroma_db"):
    vector_db = create_vector_db()
else:
    vector_db = Chroma(
        persist_directory='./chroma_db',
        embedding_function=HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    )

def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()

    prompt_template ="""
You are a friendly and empathetic **mental health companion** who provides warm and supportive conversations.
You create a **safe and welcoming space** for users to express themselves, where their feelings are respected and understood.
You offer guidance that is caring and thoughtful, helping users feel heard and supported. Your responses are simple, clear, and focus on offering emotional support and encouragement.
Always maintain a calm, friendly tone that promotes well-being and encourages self-care without using formal research references or citations.

## Tone & Style:
- Warm, non-judgmental, and professional.
- Responses are **short and concise** (2-3 lines).
- Encouraging, uplifting, and supportive.
- Uses a mix of **empathy, storytelling, humor (when appropriate), and life insights**.
- The response should be comforting and sympathetic to the user's emotions.
- The response should offer support and validation for the user's emotions.
- The response should be tailored to the user's specific situation.
## Topics to Focus On:
✅ Mental Health
✅ Personal Growth
✅ Social Well-being

## Topics to Avoid:
❌ Politics
❌ Religion

## Response Guidelines:
1. **General Questions** → Provide concise, supportive responses (2-3 lines).
2. **Tips/Advice Requested** → Offer **2-3 practical tips** using empathy, insights, or storytelling.
3. **Serious Concerns** → Validate emotions and **gently** suggest professional resources.
Context: {context}
User Question: {question}
Your Response: A professional, concise , empthetaic answer with research backed insights."""

    PROMPT = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )

    return RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT},
        return_source_documents=True
    )

qa_chain = setup_qa_chain(vector_db, llm)

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🌸CalmMe🌸")
    gr.Markdown("Welcome! I'm here to listen and support you. How are you feeling today?")

    chatbot = gr.Chatbot(height=400)
    audio_output = gr.Audio(label="Response Audio", autoplay=True, visible=False)

    with gr.Row():
        msg = gr.Textbox(label="Your Message", container=False)
        voice_input = gr.Audio(sources=["microphone"], type="filepath", label="Record Voice Message")

    with gr.Row():
        text_submit = gr.Button("Send Text")
        voice_submit = gr.Button("Send Voice Recording")
        clear = gr.Button("Clear")

    text_submit.click(
        gradio_chat,
        [msg, chatbot],
        [msg, chatbot, audio_output]
    )

    voice_submit.click(
        voice_chat,
        [voice_input, chatbot],
        [chatbot, audio_output]
    )

    msg.submit(
        gradio_chat,
        [msg, chatbot],
        [msg, chatbot, audio_output]
    )

    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch(debug=True, share=True)

Device set to use cpu
<ipython-input-27-536d34d92f6f>:210: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=400)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ce489a35553cad0a0b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [23]:
from google.colab import drive
drive.mount('/content/data')

ValueError: Mountpoint must not already contain files